## Stable Differentiable Causal Discovery (SDCD) Tutorial

In this brief tutorial, we will go over the main API for the `sdcd` package. SDCD is a differentiable causal discovery method designed to scale reliably to hundreds and thousands of variables. The algorithm is implemented in vanilla PyTorch and also uses the `networkx` package. 

(Note, under the default pip installation, only the packages necessary to run the `SDCD` model are installed. If you would also like to run other causal discovery methods implemented in this package, you must also install the `benchmark` extra via the `pip install sdcd[benchmark]` command.)

In [1]:
# Imports
from sdcd.models import SDCD
from sdcd.utils import create_intervention_dataset

We will start by simulating some data for the tutorial. If you have your own data, you can ignore this step.

In [2]:
# Simulate Data
from sdcd.simulated_data import random_model_gaussian_global_variance # For demonstration

n = 200
n_per_intervention = 50
d = 20
n_edges = 20

true_causal_model = random_model_gaussian_global_variance(
    d,
    n_edges,
    dag_type="ER",
    scale=0.5,
    hard=True,
)
X_df = true_causal_model.generate_dataframe_from_all_distributions(
    n_samples_control=n,
    n_samples_per_intervention=n_per_intervention,
)
X_df.iloc[:, :-1] = (X_df.iloc[:, :-1] - X_df.iloc[:, :-1].mean()) / X_df.iloc[
    :, :-1
].std() # Normalize the data

The input data should be formatted as a Pandas Dataframe where each row corresponds to one observation and each column corresponds to one variable. There should be an additional column reports which variable(s) was intervened on for the given observation. Here it is labeled as `perturbation_label`. For rows that do not have any interventions, the value should be set to `"obs"`.

In [3]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,perturbation_label
0,0.391698,-1.224345,1.009639,0.705815,0.348071,0.825299,-0.968303,-0.122147,-1.580835,0.410114,...,0.851675,-0.464392,3.120068,0.781549,2.148383,-0.618127,-1.280245,-0.090107,1.734710,obs
1,0.051797,-0.738174,-0.911793,-0.920152,1.611169,-1.064611,-0.081493,0.684529,0.207321,-0.051149,...,-0.112386,0.903136,-0.794169,1.281150,1.143647,-0.366966,-0.972601,1.129262,-0.024321,obs
2,-0.682005,0.614947,-0.482474,0.288308,-0.446615,0.374962,1.215926,-0.607093,-0.674821,1.333666,...,-0.233998,-0.177492,-1.087773,-0.411030,-0.533553,0.958684,0.041187,-0.270466,-0.265889,obs
3,-0.096387,0.747669,1.092575,0.840153,-0.930001,0.921199,0.040558,-0.800789,0.502776,-0.326972,...,0.328454,-0.420241,-0.706956,0.738233,0.385180,-0.147232,0.282587,-0.509720,0.318171,obs
4,0.399738,0.164933,1.735580,1.176212,-1.021558,1.288918,1.316396,-0.948071,-0.505514,-0.606622,...,1.050134,-1.656988,0.800289,-0.639471,1.263645,-0.350533,-0.958890,-1.156175,0.525499,obs


Now, to construct a torch dataset with the appropriate tensors, we use the utility function `sdcd.utils.create_intervention_dataset`.
 
(Note, we also have a function `sdcd.utils.create_dataset_anndata` for users working with the AnnData data format.)

In [4]:
X_dataset = create_intervention_dataset(X_df, perturbation_colname="perturbation_label")
X_dataset

Now, we are ready to train the SDCD model. Here, we will set `finetune=True` which runs the algorithm until the final adjacency matrix is estimated and binarized, then proceeds with a final stage of training the model until it converges for the fixed, discretized adjacency matrix. If you only care about the predicted graph, you may set this to `False` to shorten the runtime.

In [5]:
model = SDCD()
model.train(X_dataset, finetune=True)

Epoch 0: loss=28.29, gamma=0.00
Epoch 100: loss=9.94, gamma=0.00
Epoch 200: loss=7.52, gamma=0.00
Epoch 300: loss=6.15, gamma=0.00
Epoch 400: loss=5.60, gamma=0.00
Epoch 500: loss=5.20, gamma=0.00
Epoch 600: loss=4.91, gamma=0.00
Epoch 700: loss=4.62, gamma=0.00
Epoch 800: loss=4.45, gamma=0.00
Epoch 900: loss=4.18, gamma=0.00
Epoch 1000: loss=3.99, gamma=0.00
Epoch 1100: loss=3.81, gamma=0.00
Epoch 1200: loss=3.69, gamma=0.00
Epoch 1300: loss=3.63, gamma=0.00
Epoch 1400: loss=3.50, gamma=0.00
Epoch 1500: loss=3.46, gamma=0.00
Epoch 1600: loss=3.41, gamma=0.00
Epoch 1700: loss=3.36, gamma=0.00
Epoch 1800: loss=3.31, gamma=0.00
Epoch 1900: loss=3.25, gamma=0.00
Fraction of possible edges in mask: 0.15
Epoch 0: loss=29.58, gamma=0.00
Epoch 100: loss=17.48, gamma=0.50
Epoch 200: loss=12.52, gamma=1.00
Epoch 300: loss=12.70, gamma=1.50
Epoch 400: loss=12.99, gamma=2.00
Epoch 500: loss=13.20, gamma=2.50
Epoch 600: loss=13.41, gamma=3.00
Epoch 700: loss=13.47, gamma=3.50
Epoch 800: loss=13.4

Lastly, we can recover the predicted adjacency matrix, both before thresholding and after thresholding, and compute likelihoods respect to a given observation.

In [6]:
adj_matrix = model.get_adjacency_matrix(threshold=True)
print(adj_matrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]


In [16]:
# We can visualize the graph with the networkx package
import networkx as nx
G = nx.from_numpy_array(adj_matrix, create_using=nx.DiGraph())
nx.draw_circular(G, with_labels=True)

In [7]:
# To retrieve the unthresholded adjacency matrix
adj_matrix = model.get_adjacency_matrix(threshold=False)
print(adj_matrix)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.9980918e-05 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 2.2248397e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.3998721e-02
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  6.0393236e-04 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 3.4711391e-04 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.8424368e-02 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00]
 [5.8926139e-03 0.0000000e+00 0.0000000e+00 0.0000000e+00 9.4088262e-01
  1.6795698e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0

In [8]:
model.compute_nll(X_dataset) # Reports average negative log-likelihood

8.566566569010417

This concludes the tutorial. Please raise an issue on the Github repo if you come across any issues, and we hope you find this package useful to your research!